# Spark DataFrame Basics

## Leyendo datos en un Dataframe

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [5]:
df = spark.read.json('people.json')

In [6]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



Vemos que asume el campo 'age' como **long**. Serái mejor que fuera un **integer**.

In [8]:
df.columns

['age', 'name']

In [9]:
# Esto devuielve un Dataframe
df.describe()

DataFrame[summary: string, age: string, name: string]

In [11]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [12]:
# Veamos como trabajamos el esquema usando StructField

from pyspark.sql.types import (StructField,StringType, 
                               IntegerType, StructType)



In [13]:
# Defino una estructura concreta con nombres de campos y tipos de datos
data_schema = [StructField('age', IntegerType(),True),
               StructField ('name', StringType(), True)]

In [14]:
# Ahora digo que la estructura de campos la tome del 'data_schema' que he definido
final_struct = StructType(fields=data_schema)

In [15]:
# Entonces ahora al leer le digo que asuma la estructura que he definido
df = spark.read.json('people.json',schema=final_struct)

In [16]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



## Veamos como trabajar con los datos del DataFrame

In [17]:
# Si quiero utilizar un dato del Dataframe, podria intentar el método tradicional
df['age']

Column<b'age'>

In [18]:
# Parece que devuelve una columna
type(df['age'])

pyspark.sql.column.Column

In [19]:
# Hay que usar el metodo 'select'
df.select('age').show()
# esto devuelve un dataframe que tiene una columna....

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [20]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [21]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [22]:
df.head(2)[0]

Row(age=None, name='Michael')

In [24]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [25]:
# Agregar columnas

df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [26]:
df.withColumn('doubleage',df['age']*2).show()

+----+-------+---------+
| age|   name|doubleage|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [27]:
# Rename
df.withColumnRenamed('age','my_newage').show()

+---------+-------+
|my_newage|   name|
+---------+-------+
|     null|Michael|
|       30|   Andy|
|       19| Justin|
+---------+-------+



## Usando SQL

In [28]:
df.createOrReplaceTempView('people')

In [29]:
results = spark.sql("SELECT * FROM people")

In [30]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [31]:
results = spark.sql("SELECT * FROM people WHERE age=30")
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

